## CBOW - Word2Vec Implementation Pytorch

In [1]:
import re
import nltk
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from nltk.corpus import webtext
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200
%matplotlib inline

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('webtext')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.


True

## Pre-Processing text Code

In [3]:
wordpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wordpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [4]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


In [8]:
# build a sample vocab
vocab = []

for fileid in webtext.fileids():
    vocab.append(webtext.raw(fileid))

### text preprocessing (Remove tags e.g HTML,Remove special characters, Remove stopwords) === Clean data

In [9]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in corpus]

vocab_size = len(word2id)
embed_size = 100
window_size = 2

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 31
Vocabulary Sample: [('the', 1), ('is', 2), ('and', 3), ('sky', 4), ('blue', 5), ('beautiful', 6), ('quick', 7), ('brown', 8), ('fox', 9), ('lazy', 10)]


### [context_words, target_word] pairs

In [10]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    X = []
    Y = []
    context_length = window_size*2
    for words in wids:
        sentence_length = len(words)
        for index, word in enumerate(words):           
            start = index - window_size
            end = index + window_size + 1
            context = [words[i] for i in range(start, end)if 0 <= i < sentence_length and i != index]
            x = sequence.pad_sequences([context], maxlen=context_length)
            X.append(x)
            Y.append(word)
    return X,Y

## CBOW (Contineous bag of Words Model architecture)

In [11]:
import torch
import torch.nn as nn
import numpy as np

class CBOW(torch.nn.Module):

    def __init__(self, inp_size , vocab_size, embedding_dim=100):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 100)
        self.activation_function1 = nn.ReLU()        
        self.linear2 = nn.Linear(100, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(torch.from_numpy(inputs).long())).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out
    
model = CBOW(window_size*2,vocab_size)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [13]:
for epoch in range(1, 1000):
    loss = 0
    i = 0
    X,Y = generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=10000)
    for x, y in zip(X,Y):
        i += 1
        optimizer.zero_grad()
        log_probs = model(x[0])
        loss = loss_function(log_probs,torch.Tensor([y]).long())
        loss.backward()
        optimizer.step()
        loss += loss.data
    print('Epoch:', epoch, '\tLoss:', loss)

Epoch: 1 	Loss: tensor(5.1038, grad_fn=<AddBackward0>)
Epoch: 2 	Loss: tensor(4.9339, grad_fn=<AddBackward0>)
Epoch: 3 	Loss: tensor(4.7704, grad_fn=<AddBackward0>)
Epoch: 4 	Loss: tensor(4.6138, grad_fn=<AddBackward0>)
Epoch: 5 	Loss: tensor(4.4486, grad_fn=<AddBackward0>)
Epoch: 6 	Loss: tensor(4.3001, grad_fn=<AddBackward0>)
Epoch: 7 	Loss: tensor(4.1443, grad_fn=<AddBackward0>)
Epoch: 8 	Loss: tensor(3.9982, grad_fn=<AddBackward0>)
Epoch: 9 	Loss: tensor(3.8371, grad_fn=<AddBackward0>)
Epoch: 10 	Loss: tensor(3.6970, grad_fn=<AddBackward0>)
Epoch: 11 	Loss: tensor(3.5633, grad_fn=<AddBackward0>)
Epoch: 12 	Loss: tensor(3.4263, grad_fn=<AddBackward0>)
Epoch: 13 	Loss: tensor(3.2965, grad_fn=<AddBackward0>)
Epoch: 14 	Loss: tensor(3.1613, grad_fn=<AddBackward0>)
Epoch: 15 	Loss: tensor(3.0402, grad_fn=<AddBackward0>)
Epoch: 16 	Loss: tensor(2.9310, grad_fn=<AddBackward0>)
Epoch: 17 	Loss: tensor(2.8105, grad_fn=<AddBackward0>)
Epoch: 18 	Loss: tensor(2.7157, grad_fn=<AddBackward0>)
E

In [25]:
weights = model.embeddings(torch.Tensor([list(range(0,vocab_size))]).long())

pd.DataFrame(weights.view(-1,100).tolist(), index=list(id2word.values())[0:]).head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
the,-1.020247,0.858016,-0.972954,1.621598,0.015694,0.071058,-0.546601,0.627173,0.070100,0.540836,-1.658709,-0.710162,0.553695,0.681404,0.939940,1.242287,0.095183,-1.436292,0.910662,0.286473,0.412264,0.236389,0.637720,-1.741466,-1.664177,0.194743,0.228008,-0.640350,0.526864,0.281240,2.378866,1.220054,-2.521966,0.165114,-0.357386,0.639706,0.945562,-2.888935,1.225117,0.340856,...,1.069930,0.774617,-0.063797,0.463545,1.626877,1.043870,-0.687107,-1.104190,1.499260,0.428856,-0.612172,-0.791734,1.190712,0.077100,0.949545,-0.929260,0.752076,-0.108310,0.605397,0.636176,-0.630588,-0.763144,-0.626585,1.092433,-0.112572,-0.522684,0.018455,0.125532,-0.902343,1.652762,1.157533,1.456480,1.167684,0.679525,0.390032,0.883022,1.013172,0.127352,-0.370458,0.407416
is,-0.469278,0.731419,-2.046814,1.107630,-0.602588,0.407658,-0.544868,-1.117358,-2.473301,0.079160,0.954797,-1.084227,-0.515376,-0.235780,-0.376261,-0.193844,-0.043524,0.727588,0.505118,1.006242,-0.607311,-0.436902,1.811525,-0.897329,-0.348212,0.015865,1.939071,0.395484,1.446937,-0.788801,0.875630,0.702702,-1.203162,0.620883,-2.845220,2.736629,-0.391041,-0.136500,0.472429,-1.558961,...,2.048724,-0.249179,-1.945006,0.344417,0.122567,0.743959,-0.539150,0.288773,0.198540,-0.402431,0.996831,-0.342212,1.165547,-0.513374,-0.188044,-0.739049,1.485368,-0.371456,0.615853,0.180818,-1.017328,-0.179646,-0.637363,0.252615,-0.068570,-0.491677,0.869759,0.058729,0.296049,0.852820,-0.529710,-0.124254,0.493743,0.997346,0.847354,1.380898,1.105859,0.468410,-1.300014,0.211676
and,1.303654,-0.400301,0.474038,1.324667,-2.266770,0.128912,-0.284075,-0.957212,-1.241336,0.697034,0.906708,-0.607447,-0.222340,-2.749931,0.071322,-0.807236,0.392371,-0.240701,0.277111,-0.416700,-0.802208,1.838970,0.603281,-0.184513,0.061163,1.311520,1.531312,-1.966784,0.910535,0.025413,-2.657902,-0.771684,0.841913,1.475178,0.801869,-0.333531,1.314127,0.111316,-0.349497,-0.301652,...,-0.805469,0.663779,0.062341,-1.293513,0.419038,1.360080,-0.428137,-1.014816,2.154513,0.037632,-1.194724,-0.565663,-0.230392,1.560470,0.745605,-1.303745,-0.616904,0.891984,-0.829454,0.009040,-0.167278,0.100520,0.025883,-0.090795,-0.806667,-0.212954,-0.571660,1.301951,-1.995423,-0.757284,-1.709719,-0.327805,-0.244655,-0.214154,0.025581,-0.820590,0.131171,0.425759,-0.504117,1.773358
sky,1.449165,0.153278,-1.509703,-0.340397,-0.808891,0.449833,-0.356767,-1.193008,0.558038,0.764705,0.095494,0.418742,0.503980,0.348790,-1.417122,-0.390010,-1.054957,0.292385,-2.709125,0.646774,-1.062240,-1.409375,-0.158662,0.855864,0.027267,-0.104874,0.945288,1.170664,1.482344,-0.349548,-0.754053,-2.300823,-1.179725,2.561830,-0.392510,2.203889,-1.327304,-0.063887,-0.411009,-1.667183,...,-0.587274,-1.464557,1.315149,-0.181798,-0.215293,-0.064679,-0.125366,1.888723,1.516137,-0.700118,-0.243861,1.494783,-0.556467,0.085412,-2.062423,-1.162731,-0.515584,-1.128738,0.951601,-0.017662,-2.747708,-0.893692,-0.329091,-0.308133,0.417533,-0.089249,-0.588559,0.202914,-0.494305,-0.191850,0.045936,2.844258,0.676560,0.412295,-0.157112,-0.010236,-0.340997,0.800338,0.298349,0.183261
blue,-0.469048,-0.312949,0.123316,1.329445,-0.551616,-0.133613,-0.911876,0.007528,2.264735,-0.722735,0.055664,-1.083289,0.455743,-0.041167,-0.014597,-0.915513,0.417387,-0.742937,-0.534138,0.256865,0.496202,0.672305,-0.081591,1.141091,0.126867,-0.233428,-0.047002,0.639444,1.844794,0.935714,2.214909,1.384562,-0.715399,1.816184,0.282828,0.029626,-1.683331,0.306919,0.286198,-0.184079,...,-1.061380,0.460302,-0.739375,1.300935,0.556676,0.095558,-0.119986,-0.871369,-0.106704,-0.253442,-0.041808,0.202927,-0.038642,0.794085,-0.859474,0.157171,1.115904,1.366879,1.360007,-0.997755,0.543524,0.206829,0.667261,-0.277421,-1.065924,1.259926,1.129960,0.334031,-1.456439,-1.293781,-0.815232,-2.548271,-0.507428

In [26]:
from sklearn.metrics.pairwise import euclidean_distances

weights = weights.view(-1,100)
distance_matrix = euclidean_distances(weights.detach().numpy())
similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:4]+1] 
                 for search_term in ['the', 'fox', 'beautiful','brown','lazy']}

similar_words

{'beautiful': ['over', 'but', 'brown'],
 'brown': ['has', 'jumps', "king's"],
 'fox': ['a', 'toast', 'brown'],
 'lazy': ['quick', 'a', 'this'],
 'the': ['is', 'over', 'quick']}